<div style="display: flex; justify-content: center; align-items: center;">
<p align="center">
<img src="Capa_c.jpeg" style="width:250px;height:250px" alt='imagem do daniel'>
</p>

<div style="margin-left: 20px;">
<h1 style="font-family: times new roman;">TechTrinity</h1>
<p style="margin-bottom: 5px;">Trabalho Final de Redes Neurais</p>
<p style="margin-bottom: 5px;">INTEGRANTES: Eric Leandro, Samuel Araujo e Daniel Carrasco </p>
<p style="margin-bottom: 5px;">DISCIPLINA: Redes Neurais e Algoritmos Genéticos</p>
<p style="margin-bottom: 5px;">MESTRE: Daniel Cassar</p>

</div>

In [11]:
'''IMPORTS'''

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

In [12]:
'''Pegando os dados'''

# Carregando o arquivo CSV para o teste
minerais_treino = pd.read_csv('Mineral_Dataset_Supplementary_Info.csv')

# Transformando o dataset em um Dataframe pandas
df_treino = pd.DataFrame(minerais_treino) 

# Mostrando o Dataframe
print("Dataset de treino:")
display(df_treino)

# Carregando o arquivo CSV para o teste
minerais_teste = pd.read_csv('Artificial_Crystals_Dataset.csv')

# Transformando o dataset em um Dataframe pandas
df_teste = pd.DataFrame(minerais_teste) 

# Mostrando o Dataframe
print("Dataset de teste:")
display(df_teste)

Dataset de treino:


,Unnamed: 0,Hardness,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
0,0,2.3,110.0,23.000000,36.666667,2.666667,82.598467,8.504133,2.146667,2.006667,1.253333,0.456803,7.666667
1,1,5.5,406.0,30.472136,9.902439,4.682927,19.813180,11.456151,2.700244,1.676829,0.868293,0.522909,0.743223
2,2,5.5,406.0,30.472464,10.410256,4.923077,20.931371,11.541405,2.753590,1.703846,0.894359,0.497498,0.781345
3,3,5.5,476.0,61.142136,11.609756,4.682927,23.659644,11.487395,2.763659,1.714634,0.848780,0.519474,1.491272
4,4,5.5,476.0,61.142464,12.205128,4.923077,24.975089,11.574251,2.820256,1.743590,0.873846,0.493887,1.567755
...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,625,3.8,46.0,9.133000,23.000000,4.000000,48.719500,9.877100,2.115000,1.905000,1.120000,0.478880,4.566500
618,626,4.5,86.0,6.674328,14.333333,5.166667,30.645954,11.862733,2.861667,1.700000,0.901667,0.487172,1.112388
619,628,4.0,38.0,7.134332,19.000000,4.000000,40.689515,11.506150,2.545000,1.765000,0.920000,0.479405,3.567166
620,629,7.5,86.0,8.841328,14.333333,5.000000,30.550687,11.543000,2.831667,1.735000,0.890000,0.489507,1.473555


Dataset de teste:


,Unnamed: 0,Formula,Crystal structure,Hardness (Mohs),allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
0,0,MnTeMoO6,tetragonal,4.50,167.000,23.907992,18.555556,5.000000,41.609136,11.693844,2.938889,1.711111,0.884444,0.477830,2.656444
1,1,MgH2,tetragonal,4.00,14.000,1.740168,4.666667,1.333333,8.773227,11.614333,1.903333,1.310000,0.680000,0.825990,0.580056
2,2,CO(NH2)2C4H6O5,monoclinic,2.50,102.000,8.511159,4.434783,3.304348,8.440584,13.176622,2.672609,1.379130,0.530870,0.713850,0.370050
3,3,GaPO4,trigonal,5.50,78.000,8.109328,13.000000,5.333333,27.448814,11.826400,2.960000,1.625000,0.813333,0.488163,1.351555
4,4,Ga3PO7,trigonal,6.50,164.000,19.921324,14.909091,5.090909,32.012361,11.255573,2.881818,1.640909,0.841818,0.483480,1.811029
5,5,LiB3O5,orthorhombic,6.50,58.000,7.650660,6.444444,4.444444,13.261239,10.930689,2.700000,1.686667,0.780000,0.479962,0.850073
6,6,BiB3O6,monoclinic,5.75,146.000,16.864992,14.600000,5.000000,33.739258,11.388810,2.866000,1.695000,0.786000,0.478464,1.686499
7,7,K2Al2B2O7,trigonal,6.00,130.000,11.871324,10.000000,4.307692,20.443453,10.198131,2.540000,1.820000,0.972308,0.489274,0.913179
8,8,Nd0.02GdLa0.16Ca4O(BO3)3,monoclinic,6.42,249.320,22.345960,13.713971,4.365237,29.432344,10.598482,2.525787,1.813894,0.992739,0.487604,1.229151
9,9,Nd0.02GdLa0.33Ca4O(BO3)3,monoclinic,6.35,259.010,23.392140,14.114986,4.343324,30.446537,10.551961,2.512578,1.819602,1.001515,0.486888,1.274776


In [13]:
'''Tratando os dados'''

# Retirando colunas desinteressantes
df_teste = df_teste.drop(['Unnamed: 0', 'Crystal structure', 'Formula'], axis=1)

# Trocando o nome de uma coluna para facilitar o codigo
df_teste.rename(columns={'Hardness (Mohs)': 'Hardness'}, inplace=True)

# Retirando valores nan
df_teste = df_teste.dropna()

# Ajustando o index
df_teste = df_teste.reset_index(drop=True)

df_teste # Mostrando nova tabela

,Hardness,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
0,4.50,167.000,23.907992,18.555556,5.000000,41.609136,11.693844,2.938889,1.711111,0.884444,0.477830,2.656444
1,4.00,14.000,1.740168,4.666667,1.333333,8.773227,11.614333,1.903333,1.310000,0.680000,0.825990,0.580056
2,2.50,102.000,8.511159,4.434783,3.304348,8.440584,13.176622,2.672609,1.379130,0.530870,0.713850,0.370050
3,5.50,78.000,8.109328,13.000000,5.333333,27.448814,11.826400,2.960000,1.625000,0.813333,0.488163,1.351555
4,6.50,164.000,19.921324,14.909091,5.090909,32.012361,11.255573,2.881818,1.640909,0.841818,0.483480,1.811029
5,6.50,58.000,7.650660,6.444444,4.444444,13.261239,10.930689,2.700000,1.686667,0.780000,0.479962,0.850073
6,5.75,146.000,16.864992,14.600000,5.000000,33.739258,11.388810,2.866000,1.695000,0.786000,0.478464,1.686499
7,6.00,130.000,11.871324,10.000000,4.307692,20.443453,10.198131,2.540000,1.820000,0.972308,0.489274,0.913179
8,6.42,249.320,22.345960,13.713971,4.365237,29.432344,10.598482,2.525787,1.813894,0.992739,0.487604,1.229151
9,6.35,259.010,23.392140,14.114986,4.343324,30.446537,10.551961,2.512578,1.819602,1.001515,0.486888,1.274776


In [15]:
'''Separando Features e Targets'''

# Definindo Features
FEATURES = ['allelectrons_Total',  'density_Total',  'allelectrons_Average',
            'val_e_Average',  'atomicweight_Average',  'ionenergy_Average',
            'el_neg_chi_Average',  'R_vdw_element_Average',  'R_cov_element_Average',
            'zaratio_Average',  'density_Average']

# Definindo Targets
TARGET = 'Hardness'

# Separando dados para o treino
X_train = df_treino[FEATURES]
Y_train = df_treino[TARGET]

# Separando dados para teste
X_teste = df_teste[FEATURES]
Y_teste = df_teste[TARGET]

In [ ]:
'''REDE NEURAL'''

class NeuralNetworkRegressor(nn.Module):
    """
    Implementa uma rede neural regressora com uma camada oculta, dropout e otimizador de Descida do Gradiente com Momento.

    Args:
        input_size (int): O tamanho da camada de entrada.
        hidden_size (int): O tamanho da camada oculta.
        dropout_prob (float): A probabilidade de dropout aplicada à camada oculta (default 0.3).
        momentum (float): O fator de momentum para o otimizador (default 0.9).
    """

    def __init__(self, input_size, hidden_size, learning_rate=0.1, dropout_prob=0.3, momentum=0.9):
        super(NeuralNetworkRegressor, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout_prob = dropout_prob
        self.momentum = momentum
        
        # Camada oculta
        self.hidden_layer = nn.Linear(input_size, hidden_size)
        self.hidden_activation = nn.Sigmoid()
        
        # Camada de saída
        self.output_layer = nn.Linear(hidden_size, 1)
        
        # Dropout
        self.dropout = nn.Dropout(p=dropout_prob)
        
        # Otimizador com Momento
        self.optimizer = optim.SGD(self.parameters(), lr=learning_rate, momentum=momentum)
        
        # Função de perda (Mean Squared Error)
        self.criterion = nn.MSELoss()
    
    def forward(self, x):
        """
        Realiza a propagação direta dos dados pela rede.

        Args:
            x (torch.Tensor): Os dados de entrada.

        Returns:
            torch.Tensor: A saída prevista pela rede.
        """
        # Camada oculta
        hidden_output = self.hidden_layer(x)
        hidden_output = self.hidden_activation(hidden_output)
        
        # Dropout (apenas durante o treinamento)
        if self.training:
            hidden_output = self.dropout(hidden_output)
        
        # Camada de saída
        output = self.output_layer(hidden_output)
        return output
    
    def train_model(self, X_train, y_train, epochs):
        """
        Treina a rede neural usando os dados fornecidos.

        Args:
            X_train (torch.Tensor): Os dados de entrada de treinamento.
            y_train (torch.Tensor): Os rótulos esperados de treinamento.
            epochs (int): O número de épocas de treinamento.
        """
        for epoch in range(epochs):
            # Ativa o modo de treinamento
            self.train()
            
            # Realiza a propagação direta
            outputs = self(X_train)
            
            # Calcula a perda
            loss = self.criterion(outputs, y_train)
            
            # Retropropagação e atualização dos pesos
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            # Mostra a perda a cada 100 épocas
            if (epoch + 1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')


In [ ]:
'''Treinando a rede'''

# Parâmetros da rede neural
input_size = 6
hidden_size = 10
epochs = 5000
learning_rate = 0.2

# Criação e treinamento da rede neural
neural_net = NeuralNetworkRegressor(input_size, hidden_size, learning_rate)
neural_net.train_model(X_train, y_train, epochs)

NameError: name 'X_train' is not defined